# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

WORK IN PROGRESS

### Imports

In [485]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Hyperparameters

In [486]:
# TODO: Set hyperparameters.
training_batch_size = 20 #1020
validation_batch_size = 20 #1020
test_batch_size = 43 #6149
epochs = 20
learning_rate = 0.001
#momentum = 0.9
crop_size = 128

### Device

In [487]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

Found 1 GPUs. Using cuda:0.


### Load Dataset

In [488]:
original_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

h_flipped_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

v_flipped_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomVerticalFlip(0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

pos90_rotate_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomRotation([89,91]),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

minus90_rotate_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomRotation([-91,-89]),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

augmented_training_data = ConcatDataset([original_training_data,h_flipped_training_data,v_flipped_training_data,pos90_rotate_training_data,minus90_rotate_training_data])

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [489]:
train_dataloader = DataLoader(augmented_training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

## Model

In [490]:
classifications = F.one_hot(torch.tensor([e for e in range(0,102)], device="cuda:0"), num_classes=102)
classifications, classifications.size()

(tensor([[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0'),
 torch.Size([102, 102]))

In [491]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(8, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) #3 inputs 6 hiddens
		#self.batchnorm1 = nn.BatchNorm2d(6) #Normalizes above 
		self.conv2 = nn.Conv2d(6, 30, 3) # 12 hiddens
		self.fc1 = nn.Linear(8112, 204)
		self.dropout = nn.Dropout2d(0.2)
		self.fc2 = nn.Linear(204, 102) #102 output neurons         a = samples/(inputs+outputs)(hiddens) = 7140/((102+3)x18) = 3.8

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		#x = self.batchnorm1(x)
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(training_batch_size, -1)
		x = self.F.relu(self.fc1(x))

		return x

net = F102Classifier()
if device == torch.device("cuda:0"):
	net.cuda()

### Loss Function & Optimiser

In [492]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=learning_rate)

## Train

In [493]:
def train(dataloader, model, loss_fn, optimizer):
	epoch = 1
	for batch, (i,j) in enumerate(dataloader):
		features, labels = i.to(device), j.to(device)
  
		# Compute the loss based off the predictions vs labels
		predictions = model(features)
		loss = loss_fn(predictions, labels)

		#Compute back propagation
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if (batch+1) % 255 == 0: #51 muliplied depending on number on augments
			print(f'Average Training Loss: {loss.item()}\n')

### Testing

In [494]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.cuda()
    model.eval()
    test_loss, correct = 0, 0
    for batch, (i,j) in enumerate(dataloader):
        features, labels = i.to(device), j.to(device)
        model.cuda()
        pred = model(features)
        test_loss += loss_fn(pred, labels).item()
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

### Running Training and Testing

In [495]:
for t in range(epochs):
    print(f'Epoch {t+1}-------------')
    train(train_dataloader, net, loss_function, optimiser)
    #print("Training Test")
    #test(train_dataloader, net, loss_function)
    print("Validation Test")
    test(validation_dataloader, net, loss_function)
print('Finished Training and Testing')

Epoch 1-------------
Average Training Loss: 4.423996925354004

Validation Test
Accuracy: 5.2%, Avg loss : 4.305999 

Epoch 2-------------
Average Training Loss: 3.0216636657714844

Validation Test
Accuracy: 16.9%, Avg loss : 3.629190 

Epoch 3-------------
Average Training Loss: 2.7804243564605713

Validation Test
Accuracy: 21.5%, Avg loss : 3.610119 

Epoch 4-------------
Average Training Loss: 1.747431755065918

Validation Test
Accuracy: 23.1%, Avg loss : 3.836395 

Epoch 5-------------
Average Training Loss: 0.9510003328323364

Validation Test
Accuracy: 22.5%, Avg loss : 4.216282 

Epoch 6-------------
Average Training Loss: 0.714943528175354

Validation Test
Accuracy: 21.2%, Avg loss : 4.564925 

Epoch 7-------------
Average Training Loss: 0.5750941038131714

Validation Test
Accuracy: 23.1%, Avg loss : 5.174266 

Epoch 8-------------
Average Training Loss: 0.18475165963172913

Validation Test
Accuracy: 23.2%, Avg loss : 5.633834 

Epoch 9-------------
Average Training Loss: 0.09506

KeyboardInterrupt: 

### Save Model

In [ ]:
""" 30,0.001 = 15.2%
 100, 0.001 = 14.3%
 50, 0.001 = 16.4%
 30,0.01 = 1.0%  Herma-OF"""

"""For Theo	

        self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) 
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)	
30, 0.01 = 1.0%
30, 0.001 = 14.0%
100, 0.001 = 13.2%
30, 0.0001 = 7.2%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.conv2 = nn.Conv2d(12, 48, 3)
		self.conv3 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(18816, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)		
100, 0.001 = 9.5%
30, 0.001 = 17.8%
50, 0.001 = 1.0%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.fc1 = nn.Linear(47628, 102)
30, 0.001 = 19.9%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 17.9%
15, 0.001 = 16.0%
10, 0.001 = 19.0%
5, 0.001 = 14.4%

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 19.2%
10, 0.001 = 16.8%
50, 0.001 = 17.0

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 102)
30, 0.001 = 18.6%

MORE
self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.fc1 = nn.Linear(21600, 102)
	def forward(self, x):
		x = (F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = self.pool(F.relu(self.conv3(x)))
		x = x.view(training_batch_size, -1)
		x = self.fc1(x)
        5 epochs = 10.5%
        10 epochs = 17.2%
        30 epochs = 16.1%
        15 epochs = 10.9
        
        Same as above but kernel size 4 and self.fc1 = nn.Linear(18816, 102)
        10 epochs = 9.2%
        15 epochs = 19.2%
        20 epochs = 21.5%
        25 epochs = 14.5%
        30 epochs = 18.4%
        
        Same as above but kernal size 8 and self.fc1 = nn.Linear(15000, 102)
        20 epochs = 15.4 %
        30 epochs = 25.5%
        35 epochs = 16.1%
        
        Same as above but with batch normilization after conv1
        20 epochs = 12.9%
        25 epochs = 25.0%
        30 epochs = 19.6%
        40 epochs = 18.0%
        
        Same as above but with dropout of 0.5 after conv1 and batch normilization
        10 epochs = 7.3%
        15 epochs = 9.1%
        20 epochs = 12.4%
        30 epochs = 10.3%
        40 epochs = 11.6%
        
        Same as above but dropout set to 0.2
        20 epochs = 10.5%
        25 epochs = 16.2%
        30 epochs = 16.7%
        32 epochs = 12.6%
        35 epochs = 19.1%
        40 epochs = 13.5%
        50 epochs = 9.3%
        
        Now have implemented testing per epoch, now will just record the peak epoch.

        Same as above but with conv3 removed and self.fc1 = nn.Linear(8112, 102)
        50 epochs = 28.0%
        
        Same as above but modified conv2 to have 6 outputs and nn.Linear(4056, 102)
        20 epochs = 20.0%
        Reverting to previous conv2 and fcl
        
        Now reverted, applying data augmentation: horizontal flip, vertical flip, 90 degree rotates, -90 degree rotates
        
        Turns out it wasn't properly applying augmentations, I'm using less exact values for the randomness so it basically always does it.
        4 epochs = 32.5%

        Same as above but without the vertical flipping
        5 epochs = 30.9%
        It seems vertical flipping is beneficial

        Same as before but with vertical flipping back and additionally 45 and -45 rotation augments
        4 epochs = 32.3%

        Same as before but removing both 45 degree rotation augments and setting dropout rate at 0.3
        3 epochs = 29%

        Same as before but setting dropout rate to 0.1
        2 epochs = 20.2%

        Same as before but setting dropout rate to 0.25
        6 epochs = 26.7%

        Restting dropout rate back to 0.2 and now testing it on the training data to see if the loss function is the issue
        6 epochs 31.4%
        Loss function is maybe fine

        same as before but removing batch normilization
        9 epochs = 31.7%
        without normilization accuracy increases slower but is more consistently high, will continue without it for now

        Same as before but pool kernal of 4 self.fc1 = nn.Linear(10092, 102)
        20 epochs = 30.8%
        Current model as follows:
        self.pool = nn.MaxPool2d(8, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		#self.batchnorm1 = nn.BatchNorm2d(6)
		self.dropout = nn.Dropout2d(0.2)
		self.conv2 = nn.Conv2d(6, 12, 3)
		#self.conv3 = nn.Conv2d(3, 24, 3)
		self.fc1 = nn.Linear(8112, 102)
	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		#x = self.batchnorm1(x)
		x = self.dropout(x)
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(training_batch_size, -1)
		x = self.fc1(x)

        Massive change



        
 """
save_path = "./models/classifier.pth"
torch.save(net.state_dict(), save_path)